In [1]:
import os
import random
import pandas as pd
import gmaps
import requests 
import json
import time
import numpy as np
from us import states
from config import (gkey)


# Set path for file
csvpath = os.path.join("loan.csv")
print(csvpath)

loan.csv


In [2]:
loans_df = pd.read_csv(csvpath)
loans_df.head()

C:\Users\Ryanj\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
pertinent_loans_df = loans_df[[ "annual_inc", "loan_amnt", "term", "int_rate", "total_pymnt", "issue_d", "addr_state", "zip_code"]]
pertinent_loans_df

,annual_inc,loan_amnt,term,int_rate,total_pymnt,issue_d,addr_state,zip_code
0,55000.0,2500,36 months,13.56,167.020000,Dec-2018,NY,109xx
1,90000.0,30000,60 months,18.94,1507.110000,Dec-2018,LA,713xx
2,59280.0,5000,36 months,17.97,353.890000,Dec-2018,MI,490xx
3,92000.0,4000,36 months,18.94,286.710000,Dec-2018,WA,985xx
4,57250.0,30000,60 months,16.14,1423.210000,Dec-2018,MD,212xx
...,...,...,...,...,...,...,...,...
2260663,58000.0,12000,60 months,14.08,5360.960000,Oct-2017,VT,054xx
2260664,30000.0,12000,60 months,25.82,14499.802172,Oct-2017,OR,971xx
2260665,64000.0,10000,36 months,11.99,5306.940000,Oct-2017,IL,603xx
2260666,60000.0,12000,60 months,21.45,5207.290000,Oct-2017,AK,996xx


In [4]:
pertinent_loans_df["annual_inc"].max()

110000000.0

In [5]:
totalNumberOfLoans = pertinent_loans_df["annual_inc"].count()
print(totalNumberOfLoans)

2260664


In [6]:
# Create the bins in which annual_inc will be held, based on tax brackets
bins = [0, 9700, 39475, 84200, 160725, 204100, 510300, 110000000]


# Create the names for the four bins
income_tiers = ["Tier_One", "Tier_Two", "Tier_Three", "Tier_Four", "Tier_Five", "Tier_Six", "Tier_Seven"]

In [7]:
a = pd.cut(pertinent_loans_df["annual_inc"], bins, labels=income_tiers)
a.head()

0    Tier_Three
1     Tier_Four
2    Tier_Three
3     Tier_Four
4    Tier_Three
Name: annual_inc, dtype: category
Categories (7, object): [Tier_One < Tier_Two < Tier_Three < Tier_Four < Tier_Five < Tier_Six < Tier_Seven]

In [8]:
pertinent_loans_df["tier_group"] = a
pertinent_loans_df.head()

C:\Users\Ryanj\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,annual_inc,loan_amnt,term,int_rate,total_pymnt,issue_d,addr_state,zip_code,tier_group
0,55000.0,2500,36 months,13.56,167.02,Dec-2018,NY,109xx,Tier_Three
1,90000.0,30000,60 months,18.94,1507.11,Dec-2018,LA,713xx,Tier_Four
2,59280.0,5000,36 months,17.97,353.89,Dec-2018,MI,490xx,Tier_Three
3,92000.0,4000,36 months,18.94,286.71,Dec-2018,WA,985xx,Tier_Four
4,57250.0,30000,60 months,16.14,1423.21,Dec-2018,MD,212xx,Tier_Three


In [9]:
# Create a GroupBy object based upon "View Group"
loan_group = pertinent_loans_df.groupby("tier_group")

# Find how many rows fall into each bin
print(loan_group["annual_inc"].count())

# Get the average of each column within the GroupBy object
loan_group[["int_rate", "loan_amnt", "annual_inc"]].mean()

tier_group
Tier_One         2954
Tier_Two       343095
Tier_Three    1196274
Tier_Four      604591
Tier_Five       59806
Tier_Six        48894
Tier_Seven       3383
Name: annual_inc, dtype: int64


,int_rate,loan_amnt,annual_inc
tier_group,,,
Tier_One,14.813074,11096.267773,6.529720e+03
Tier_Two,13.962959,8677.464842,2.989422e+04
Tier_Three,13.252005,13711.771613,5.970080e+04
Tier_Four,12.506046,19535.365230,1.101832e+05
Tier_Five,11.945691,23553.136809,1.814288e+05
Tier_Six,11.709249,25823.841371,2.769104e+05
Tier_Seven,11.682722,26295.876441,1.068499e+06


In [10]:
sampleOne = pertinent_loans_df.sample(1000)
sampleOne.head()

,annual_inc,loan_amnt,term,int_rate,total_pymnt,issue_d,addr_state,zip_code,tier_group
2071303,110000.0,20000,36 months,17.57,2838.700000,Feb-2014,MI,488xx,Tier_Four
1339284,85000.0,20000,60 months,14.31,23704.610000,Jan-2015,IN,473xx,Tier_Four
61827,65000.0,5000,36 months,15.02,661.800000,Nov-2018,AL,368xx,Tier_Three
1051089,110500.0,28000,36 months,7.26,30867.108182,Oct-2015,AK,997xx,Tier_Four
1571946,100000.0,12000,36 months,9.44,7668.610000,Jun-2017,CA,958xx,Tier_Four


Create a DataFrame for each tier

In [ ]:
Tier_One_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_One"] 

#sample test group of 100
tierOneSample = Tier_One_df.sample(100)


tierOne_totalLoans = Tier_One_df["annual_inc"].count()

tierOne_averageLoanAmount = Tier_One_df["loan_amnt"].mean()
tierOne_averageLoanAmount = round(tierOne_averageLoanAmount)

tierOne_averageInterestRate = Tier_One_df["int_rate"].mean()
tierOne_averageInterestRate = format(tierOne_averageInterestRate, "4.2f")

print("Total number of loans for tier one: " + str(tierOne_totalLoans))
print("Average loan ammount: " + str(tierOne_lone_amnt_mean))
print("Average interest rate: " + str(tierOne_averageInterestRate))


In [12]:
Tier_One_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_One"] 

#sample test group of 100
tierOneSample = Tier_One_df.sample(100)


tierOne_totalLoans = Tier_One_df["annual_inc"].count()

tierOne_averageLoanAmount = Tier_One_df["loan_amnt"].mean()
tierOne_averageLoanAmount = round(tierOne_averageLoanAmount)

tierOne_averageInterestRate = Tier_One_df["int_rate"].mean()
tierOne_averageInterestRate = format(tierOne_averageInterestRate, "4.2f")

print("Total number of loans for tier one: " + str(tierOne_totalLoans))
print("Average loan ammount: " + str(tierOne_averageLoanAmount))
print("Average interest rate: " + str(tierOne_averageInterestRate))


Total number of loans for tier one: 2954
Average loan ammount: 11096
Average interest rate: 14.81


In [13]:
Tier_Two_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_Two"] 
#sample
tierTwoSample = Tier_Two_df.sample(100)
Tier_Two_df["annual_inc"].count()

343095

In [14]:
Tier_Three_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_Three"] 
#sample
tierThreeSample = Tier_Three_df.sample(100)
Tier_Three_df["annual_inc"].count()

1196274

In [15]:
Tier_Four_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_Four"] 
#sample
tierFourSample = Tier_Four_df.sample(100)
Tier_Four_df["annual_inc"].count()

604591

In [16]:
Tier_Five_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_Five"] 
#sample
tierFiveSample = Tier_Five_df.sample(100)
Tier_Five_df["annual_inc"].count()

59806

In [17]:
Tier_Six_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_Six"] 
#sample
tierSixSample = Tier_Six_df.sample(100)
Tier_Six_df["annual_inc"].count()

48894

In [18]:
Tier_Seven_df = pertinent_loans_df[pertinent_loans_df["tier_group"] == "Tier_Seven"] 
#sample
tierSevenSample = Tier_Seven_df.sample(100)
Tier_Seven_df["annual_inc"].count()

3383

Heat Map

In [19]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [20]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
coordinates_df = pd.read_csv(
    "Output/coordinates.csv", dtype="object", encoding="utf-8")

# Visualize
coordinates_df.head()

,addr_state,Latitudinal Coordinates,Longitudinal Coordinates
0,AL,32.3792233,-86.3077368
1,AK,58.3019444,-134.4197221
2,AZ,33.4483771,-112.0740373
3,AR,34.7464809,-92.28959479999999
4,CA,38.5815719,-121.4943996


In [21]:
# Merge the two data sets along zip code
pertinent_loans_coordinates_df = pd.merge(
    pertinent_loans_df, coordinates_df, how="left", on=["addr_state", "addr_state"])

# Remove rows missing data
pertinent_loans_coordinates_df = pertinent_loans_coordinates_df.dropna()

# Visualize
pertinent_loans_coordinates_df.head()

,annual_inc,loan_amnt,term,int_rate,total_pymnt,issue_d,addr_state,zip_code,tier_group,Latitudinal Coordinates,Longitudinal Coordinates
0,55000.0,2500,36 months,13.56,167.02,Dec-2018,NY,109xx,Tier_Three,42.6525793,-73.7562317
1,90000.0,30000,60 months,18.94,1507.11,Dec-2018,LA,713xx,Tier_Four,30.4514677,-91.18714659999999
2,59280.0,5000,36 months,17.97,353.89,Dec-2018,MI,490xx,Tier_Three,42.732535,-84.5555347
3,92000.0,4000,36 months,18.94,286.71,Dec-2018,WA,985xx,Tier_Four,47.0378741,-122.9006951
4,57250.0,30000,60 months,16.14,1423.21,Dec-2018,MD,212xx,Tier_Three,38.9784453,-76.4921829


In [22]:
# Create a GroupBy object
loan_state = pertinent_loans_coordinates_df.groupby("addr_state")

# Find how many rows fall into each bin
loansPerState = loan_state["annual_inc"].count()
loansPerState = list(loansPerState)
print(loansPerState)

max(loansPerState)

[5227, 27264, 17054, 53711, 314299, 48140, 35771, 6455, 161893, 74153, 10663, 14, 4295, 91113, 37480, 19089, 21872, 25737, 51763, 53987, 4972, 58745, 39501, 36052, 12633, 6293, 62683, 3585, 7810, 11137, 83092, 11970, 32616, 186324, 75073, 20672, 26760, 76882, 10002, 27983, 4548, 35445, 186144, 14964, 62908, 4936, 46991, 29851, 8345, 4743]


314299

In [23]:
# Get the average of each column within the GroupBy object
#loan_state[["int_rate", "loan_amnt", "annual_inc"]].mean()


In [24]:
totalNumberOfLoans = sum(loansPerState)
print(totalNumberOfLoans)

2253640


In [25]:
stateLoansVStotal_list = []

In [26]:
for i in loansPerState:
    ratio = i/totalNumberOfLoans
    stateLoansVStotal_list.append(ratio)
sum(stateLoansVStotal_list)

1.0000000000000002

In [27]:
coordinates_df["Loan Ratio"] = stateLoansVStotal_list
coordinates_df.head()

,addr_state,Latitudinal Coordinates,Longitudinal Coordinates,Loan Ratio
0,AL,32.3792233,-86.3077368,0.002319
1,AK,58.3019444,-134.4197221,0.012098
2,AZ,33.4483771,-112.0740373,0.007567
3,AR,34.7464809,-92.28959479999999,0.023833
4,CA,38.5815719,-121.4943996,0.139463


In [28]:
coordinates_df.nlargest(5, ['Loan Ratio'])

,addr_state,Latitudinal Coordinates,Longitudinal Coordinates,Loan Ratio
4,CA,38.5815719,-121.4943996,0.139463
33,ND,46.8083268,-100.7837392,0.082677
42,TX,30.267153,-97.7430608,0.082597
8,FL,30.4382559,-84.28073289999999,0.071836
13,IN,39.768403,-86.158068,0.040429


In [29]:
coordinates_df.nsmallest(5, ['Loan Ratio'])

,addr_state,Latitudinal Coordinates,Longitudinal Coordinates,Loan Ratio
11,ID,43.6150186,-116.2023137,0.000006
27,NV,39.1637984,-119.7674034,0.001591
12,IL,39.78172130000001,-89.6501481,0.001906
40,SD,44.36678759999999,-100.3537522,0.002018
49,WY,41.1399814,-104.8202462,0.002105


In [30]:
# Store 'Lat' and 'Lng' into  locations 
locations = coordinates_df[["Latitudinal Coordinates", "Longitudinal Coordinates"]].astype(float)

In [31]:
# Convert loan Rate to float and store
loan_ratio = coordinates_df["Loan Ratio"].astype(float)

In [35]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=(loan_ratio * 3000), 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))